# Semantic Segmentation (S3DIS)

### Jupyter Notebook implementation

First read the [README](README.md) file if you're new.

This is an example of using the code from Jupyter Notebook.

## Contents

- [Data Preparation](#data-preparation)
- [Training](#training)
- [Testing](#testing)

## Imports

In [ ]:
from pprint import pprint
from pathlib import Path

import torch
print(f'CUDA available?: {torch.cuda.is_available()}')

import train_semseg
import test_semseg
from data_utils.collect_indoor3d_data import collect_indoor3d_data

## **Optional**: External Directory

If you want to put the log and data directories in a different location, you can use the following code.

In [ ]:
root_log_dir = Path('..', 'log').resolve()
DATA_DIR = str(Path('..', 'data').resolve())
semseg_raw_data_dir = str(Path(DATA_DIR, 's3dis', 'Stanford3dDataset_v1.2_Aligned_Version').resolve())
semseg_data_dir = str(Path(DATA_DIR, 'stanford_indoor3d').resolve())

root_log_dir, DATA_DIR, semseg_raw_data_dir, semseg_data_dir

---

## Data

- S3DIS dataset in text format
- Download `Stanford3dDataset_v1.2.zip` from:
  - [https://cvg-data.inf.ethz.ch/s3dis/](https://cvg-data.inf.ethz.ch/s3dis/)

Default `--data_dir` is `'data/s3dis/Stanford3dDataset_v1.2_Aligned_Version'`.

- `data/`
  - `s3dis/`
    - `ReadMe.txt`
    - `Stanford3dDataset_v1.2_Aligned_Version/`
      - `Area_1/`
        - `conferenceRoom_1/`
          - `Annotations/`
            - `beam_1.txt`
            - ...
          - `conferenceRoom_1.txt`
        - ...
      - ...

### **OUTPUT**

- TRAINING: `<log_root>/semantic_segmentation/<args.log_dir or TIME>/checkpoints/best_model.pth`
- PREDICTION: `None (only prints accuracy)`


---

## Data preparation

Use `data_utils/collect_indoor3d_data.py` to convert the dataset.

This will create a new directory `data/stanford_indoor3d/` (or whatever you specify with argument).

In [ ]:
!python3 data_utils/collect_indoor3d_data.py -h

```bash
python3 data_utils/collect_inddoor3d_data.py \
    --data_path data/s3dis/Stanford3dDataset_v1.2_Alligned_Version \
    --output_path data/stanford_indoor3d \
    # --skip_existing
```

In [ ]:
collect_indoor3d_data(
    data_path     = semseg_raw_data_dir,
    output_path   = semseg_data_dir,
    skip_existing = False,
)

---

## Training

`train_semseg.py` is used to train the model.

Check all the arguments:

In [ ]:
!python3 train_semseg.py -h

The following is the same as running:

```bash
python3 train_semseg.py \
     --model pointnet2_sem_seg \
     --test_area 5 \
     --log_dir pointnet2_sem_seg
```

In [ ]:
args = {
    'model'    : 'pointnet2_sem_seg',
    'test_area': 5,
    'log_dir'  : 'pointnet2_sem_seg',
    'log_root' : root_log_dir,
    'data_dir' : semseg_data_dir,
    'notebook' : True
}
semseg_train_args = train_semseg.CommandLineArgs(**args)
train_semseg.main(semseg_train_args)

---

## Testing

`test_semseg.py` is used to test the model.

Check all the arguments:

In [ ]:
!python test_semseg.py -h

The following is the same as running:

```bash
python3 test_semseg.py \
     --log_dir pointnet2_sem_seg \
     --test_area 5 \
     --visual \
     # --log_root ../log \
     # --data_dir ../data/stanford_indoor3d
```

In [ ]:
args = {
    'log_dir'  : 'pointnet2_sem_seg',
    'test_area': 5,
    'visual'   : True, # in source: action='store_true'
    'log_root' : root_log_dir,
    'data_dir' : semseg_data_dir,
    'notebook' : True
}
semseg_test_args = test_semseg.CommandLineArgs(**args)
test_semseg.main(semseg_test_args)